# Welcome to the first example notebook for Terrier-Spark

This notebook requires [Apache-Toree](https://toree.apache.org/) to be installed and working. You will also need your Terrier index, and the various topics and qrels to be set appropriately.

Firstly, lets import Terrier and Terrier-Spark using the `%AddDeps` magic command. Toree can use Maven Central for this, as well as your local Maven repository.

In [1]:
%AddDeps org.terrier terrier-core 5.0 --repository file:/home/user/.m2/repository --transitive --exclude org.slf4j:slf4j-log4j12  
%AddDeps org.terrier terrier-spark 0.0.1-SNAPSHOT --repository file:/home/user/.m2/repository --transitive



Marking org.terrier:terrier-core:5.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir9121027855058483540/toree_add_deps/
-> file:/home/user/.m2/repository
-> https://repo1.maven.org/maven2
-> New file at /home/user/.m2/repository/org/codehaus/jackson/jackson-mapper-asl/1.9.13/jackson-mapper-asl-1.9.13.pom.lastUpdated


Lets check that we have Terrier correctly imported. Do you get the correct version number?

In [2]:
org.terrier.Version.VERSION

5.0

Now lets setup the location appropriate for our local setting.

In [3]:
val terrierHome = "/home/user/projects/tr5/Terrier"
val wt2gIndex = "/home/user/projects/tr5/terrier-core-4.2-SNAPSHOT/var/index_wt2g/data.properties"
val topicsFile = "file:/home/user/TopicsQrels/TREC/WT2G/small_web/topics.401-450"
val qrelsFile = "file:/home/user/TopicsQrels/TREC/WT2G/small_web/qrels.trec8"

terrierHome = /home/user/projects/tr5/Terrier
wt2gIndex = /home/user/projects/tr5/terrier-core-4.2-SNAPSHOT/var/index_wt2g/data.properties
topicsFile = file:/home/user/TopicsQrels/TREC/WT2G/small_web/topics.401-450
qrelsFile = file:/home/user/TopicsQrels/TREC/WT2G/small_web/qrels.trec8


file:/home/user/TopicsQrels/TREC/WT2G/small_web/qrels.trec8

This is our simple function to obtain a Terrier run. It returns a Dataset with three columns: the qid, the query, and the resulting nDCG@20


In [4]:

import org.terrier.spark.ml._
import org.terrier.spark._
import org.terrier.querying.IndexRef

import org.apache.spark.sql.Row
import org.apache.spark.sql.types.{StructField,StructType,IntegerType, DoubleType}

def ndcgForModel(model : String) = {
    
    val indexref = IndexRef.of(wt2gIndex)
    
    val props = Map(
    "terrier.home" -> terrierHome,
    "termpipelines"->"")
    
    TopicSource.configureTerrier(props)
    val topics = TopicSource.extractTRECTopics(topicsFile)
        .toList.toDF("qid", "query")

    val queryTransform = new QueryingTransformer()
        .setTerrierProperties(props)
        .setIndexReference(indexref)
        .setSampleModel(model)

    val r1 = queryTransform.transform(topics)
    //r1 is a dataframe with results for queries in topics
    val qrelTransform = new QrelTransformer()
        .setQrelsFile(qrelsFile)

    val r2 = qrelTransform.transform(r1)
    //r2 is a dataframe as r1, but also includes a label column 
    val ndcg = new RankingEvaluator(Measure.NDCG, 20).evaluateByQuery(r2).toList
    
    
    val newSchema = StructType(topics.schema.fields ++ Array(StructField("ndcg", DoubleType, false)))
    spark.createDataFrame(topics.rdd.zipWithIndex.map{ case (row, index) => Row.fromSeq(row.toSeq ++ Array(ndcg(index.toInt)))}, newSchema)
}


ndcgForModel: (model: String)org.apache.spark.sql.DataFrame


Lets get the results for two weighting models, TF_IDF and BM25. What is returned is a Dataset with three columns ("qid", "query", "ndcg")

In [5]:

val TFIDF = ndcgForModel("TF_IDF")
val BM25 = ndcgForModel("BM25")

TFIDF.columns


 Assuming the value of terrier.home from the corresponding system property.
Please ensure that the property terrier.home
is specified in the file terrier.properties,
or as a system property in the command line.
TERRIER_HOME=/home/user/projects/tr5/Terrier
terrier.etc=null
TERRIER_HOME=/home/user/projects/tr5/Terrier
TERRIER_ETC=/home/user/projects/tr5/Terrier/etc
[Stage 0:=============================>                             (1 + 1) / 2]Querying /home/user/projects/tr5/terrier-core-4.2-SNAPSHOT/var/index_wt2g/data.properties for 50 queries
[Stage 2:=============================>                             (1 + 1) / 2]Got for 48131 results total
We have 47506 qrels
TERRIER_HOME=/home/user/projects/tr5/Terrier
terrier.etc=null
TERRIER_HOME=/home/user/projects/tr5/Terrier
TERRIER_ETC=/home/user/projects/tr5/Terrier/etc
Querying /home/user/projects/tr5/terrier-core-4.2-SNAPSHOT/var/index_wt2g/data.properties for 50 queries
[Stage 14:=============================>                     

TFIDF = [qid: string, query: string ... 1 more field]
BM25 = [qid: string, query: string ... 1 more field]


[qid, query, ndcg]

How did we do? Lets look at the mean NDCGs.


In [6]:

import org.apache.spark.sql.functions._

println("TFIDF")
TFIDF.select(avg($"ndcg")).show()

println("BM25")
BM25.select(avg($"ndcg")).show()

TFIDF
[Stage 24:=============================>                            (1 + 1) / 2]+------------------+
|         avg(ndcg)|
+------------------+
|0.3397919431818957|
+------------------+

BM25
+------------------+
|         avg(ndcg)|
+------------------+
|0.3386923263003945|
+------------------+



Lets look at the results side by side, so we can see where we went wrong?

In [7]:


import org.apache.spark.sql.functions.col

val joined = TFIDF
    .join(BM25, TFIDF.col("qid") === BM25.col("qid"))
    .toDF("qid", "query", "TFIDF", "qidBM25", "queryBM25", "BM25")
    .select($"qid", $"query", $"BM25" - $"TFIDF")
    .orderBy($"BM25" - $"TFIDF")

joined.filter(row => row.get(2) != 0.0).collect


[Stage 33:================================================>     (178 + 2) / 200]

joined = [qid: string, query: string ... 1 more field]


427,uv damage eyes,-0.05857478287366785
424,suicides,-0.04259925243625749
445,women clergy,-0.0420222350661259
408,tropical storms,-0.010144724844642017
401,foreign minorities germany,-0.003744521822722807
421,industrial waste disposal,-0.003600929080181753
422,art stolen forged,-0.0033479627985089078
411,salvaging shipwreck treasure,-0.002034006395408086
403,osteoporosis,-0.0016673813718870445
404,ireland peace talks,-9.332068582119102E-4
436,railway accidents,-8.390389246867025E-4


OK, so we can see which queries BM25 helped or not. Can we show this graphically? 

We will use the [vegas-viz](https://github.com/vegas-viz/Vegas) module for making the graphs, which we import using `%AddDeps` magic.

In [11]:
%AddDeps org.vegas-viz vegas_2.11 0.3.11 --transitive --exclude org.scala-lang:* --exclude org.scala-lang.modules:*
%AddDeps org.vegas-viz vegas-spark_2.11 0.3.11 --transitive --exclude org.scala-lang:* --exclude org.scala-lang.modules:*



Marking org.vegas-viz:vegas_2.11:0.3.11 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir9121027855058483540/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir9121027855058483540/toree_add_deps/https/repo1.maven.org/maven2/io/circe/circe-core_2.11/0.7.0/circe-core_2.11-0.7.0.pom
Marking org.vegas-viz:vegas-spark_2.11:0.3.11 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir9121027855058483540/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir9121027855058483540/toree_add_deps/https/repo1.maven.org/maven2/io/circe/circe-core_2.11/0.7.0/circe-core_2.11-0.7.0.pom


In [12]:
import vegas._
import vegas.render.WindowRenderer._
import vegas.sparkExt._
val queryCount = joined.count

val plot = Vegas("Per Query delta NDCG")
   .withDataFrame(joined.orderBy($"BM25" - $"TFIDF"))
   .configCell(width=600, height=300)
   .encodeX("qid", Nominal, sortOrder=SortOrder.None)
   .encodeY("(BM25 - TFIDF)", Quant)
   .mark(Bar)
plot.show

[Stage 57:>                                                        (0 + 0) / 21]

<iframe id="frame-vegas-128e61f0-b6bd-4e07-a3e0-0017266d9ab4" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-128e61f0-b6bd-4e07-a3e0-0017266d9ab4'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "bar",
 "encoding" : {
 "x" : {
 "sort" : "none",
 "field" : "qid",
 "type" : "nominal"
 },
 "y" : {
 "field" : "(BM25 - TFIDF)",
 "type" : "quantitative"
 }
 },
 "description" : "Per Query delta NDCG",
 "data" : {
 "values" : [
 {
 "qid" : "427",
 "query" : "uv damage eyes",
 "(BM25 - TFIDF)" : -0.05857478287366785
 },
 {
 "qid" : "424",
 "query" : "suicides",
 "(BM25 - TFIDF)" : -0.04259925243625749
 },
 {
 "qid" : "445",
 "query" : "women clergy",
 "(BM25 - TFIDF)" : -0.0420222350661259
 },
 {
 "qid" : "408",
 "query" : "tropical storms",
 "(BM25 - TFIDF)" : -0.010144724844642017
 },
 {
 "qid" : "401",
 "query" : "foreign minorities germany",
 "(BM25 - TFIDF)" : -0.003744521822722807
 },
 {
 "qid" : "421",
 "query" : "industrial waste disposal",
 "(BM25 - TFIDF)" : -0.003600929080181753
 },
 {
 "qid" : "422",
 "query" : "art stolen forged",
 "(BM25 - TFIDF)" : -0.0033479627985089078
 },
 {
 "qid" : "411",
 "query" : "salvaging shipwreck treasure",
 "(BM25 - TFIDF)" : -0.002034006395408086
 },
 {
 "qid" : "403",
 "query" : "osteoporosis",
 "(BM25 - TFIDF)" : -0.0016673813718870445
 },
 {
 "qid" : "404",
 "query" : "ireland peace talks",
 "(BM25 - TFIDF)" : -9.332068582119102E-4
 },
 {
 "qid" : "436",
 "query" : "railway accidents",
 "(BM25 - TFIDF)" : -8.390389246867025E-4
 },
 {
 "qid" : "412",
 "query" : "airport security",
 "(BM25 - TFIDF)" : -5.916538929868764E-4
 },
 {
 "qid" : "447",
 "query" : "stirling engine",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "415",
 "query" : "drugs golden triangle",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "434",
 "query" : "estonia economy",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "423",
 "query" : "milosevic mirjana markovic",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "440",
 "query" : "child labor",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "413",
 "query" : "steel production",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "450",
 "query" : "king hussein peace",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "430",
 "query" : "killer bee attacks",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "444",
 "query" : "supercritical fluids",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "420",
 "query" : "carbon monoxide poisoning",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "435",
 "query" : "curbing population growth",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "442",
 "query" : "heroic acts",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "428",
 "query" : "declining birth rates",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "425",
 "query" : "counterfeiting money",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "410",
 "query" : "schengen agreement",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "449",
 "query" : "antibiotics ineffectiveness",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "429",
 "query" : "legionnaires disease",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "439",
 "query" : "inventions scientific discoveries",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "414",
 "query" : "cuba sugar exports",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "407",
 "query" : "poaching wildlife preserves",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "409",
 "query" : "legal pan am 103",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "426",
 "query" : "law enforcement dogs",
 "(BM25 - TFIDF)" : 0.0
 },
 {
 "qid" : "406",
 "query" : "parkinson s dis

queryCount = 50
plot = ExtendedUnitSpecBuilder(ExtendedUnitSpec(None,None,Bar,Some(Encoding(None,None,Some(PositionChannelDef(None,None,Some(SortSortOrder(None)),Some(qid),Some(Nominal),None,None,None,None,None)),Some(PositionChannelDef(None,None,None,Some((BM25 - TFIDF)),Some(Quantitative),None,None,None,None,None)),None,None,None,None,None,None,None,None,None,None,None)),None,Some(Per Query delta NDCG),Some(Data(None,None,Some(List(Values(Map(qid -> 427, query -> uv damage eyes, (BM25 - TFIDF) -> -0.05857478287366785)), Values(Map(qid -> 424, query -> suicides, (BM25 - TFIDF) -> -0.04259925243625749)), Values(Map(qid -> 445, query -> women clergy, (BM25 - TFIDF) -> -0.0...


ExtendedUnitSpecBuilder(ExtendedUnitSpec(None,None,Bar,Some(Encoding(None,None,Some(PositionChannelDef(None,None,Some(SortSortOrder(None)),Some(qid),Some(Nominal),None,None,None,None,None)),Some(PositionChannelDef(None,None,None,Some((BM25 - TFIDF)),Some(Quantitative),None,None,None,None,None)),None,None,None,None,None,None,None,None,None,None,None)),None,Some(Per Query delta NDCG),Some(Data(None,None,Some(List(Values(Map(qid -> 427, query -> uv damage eyes, (BM25 - TFIDF) -> -0.05857478287366785)), Values(Map(qid -> 424, query -> suicides, (BM25 - TFIDF) -> -0.04259925243625749)), Values(Map(qid -> 445, query -> women clergy, (BM25 - TFIDF) -> -0.0420222350661259)), Values(Map(qid -> 408, query -> tropical storms, (BM25 - TFIDF) -> -0.010144724844642017)), Values(Map(qid -> 401, query -> foreign minorities germany, (BM25 - TFIDF) -> -0.003744521822722807)), Values(Map(qid -> 421, query -> industrial waste disposal, (BM25 - TFIDF) -> -0.003600929080181753)), Values(Map(qid -> 422, quer

And there we have it - query 419 was most benefitted by BM25, while 424 and 427 were least.